In [ ]:
! pip install bertopic



     |████████████████████████████████| 57 kB 2.2 MB/s 
     |████████████████████████████████| 86 kB 3.3 MB/s 
     |████████████████████████████████| 79 kB 6.0 MB/s 
     |████████████████████████████████| 5.2 MB 39.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.8 MB 47.8 MB/s 
     |████████████████████████████████| 1.2 MB 45.0 MB/s 
     |████████████████████████████████| 67 kB 4.1 MB/s 
     |████████████████████████████████| 636 kB 49.5 MB/s 
     |████████████████████████████████| 6.5 MB 44.0 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 1.1 MB 46.8 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2330779 sha256=8ad9f35ba2cf230033285924d60afdde1736ac0bd6a47579db648c49896a02e3
  Stored in directory: /root/.cache/pip/wheels/6e/7a/5e/259ccc841c085fc41b99ef

In [ ]:

from bertopic import BERTopic
import pandas as pd



In [ ]:
import pandas as pd
import re
import gensim
from nltk.stem import WordNetLemmatizer

punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'         # define a string of punctuation symbols

# Functions to clean tweets
def remove_links(tweet):
    """Takes a string and removes web links from it"""
    tweet = re.sub(r'http\S+', '', tweet)   # remove http links
    tweet = re.sub(r'bit.ly/\S+', '', tweet)  # remove bitly links
    tweet = tweet.strip('[link]')   # remove [links]
    tweet = re.sub(r'pic.twitter\S+','', tweet)
    return tweet

def remove_users(tweet):
    """Takes a string and removes retweet and @user information"""
   # tweet = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', tweet)  # remove re-tweet
    tweet = re.sub('(@)', '', tweet)  # remove tweeted at
    return tweet

def remove_hashtags(tweet):
    """Takes a string and removes any hash tags"""
    tweet = re.sub('(#)', ' ', tweet)  # remove hash tags
    return tweet

def remove_av(tweet):
    """Takes a string and removes AUDIO/VIDEO tags or labels"""
    tweet = re.sub('VIDEO:', '', tweet)  # remove 'VIDEO:' from start of tweet
    tweet = re.sub('AUDIO:', '', tweet)  # remove 'AUDIO:' from start of tweet
    return tweet

def tokenize(tweet):
    """Returns tokenized representation of words in lemma form excluding stopwords"""
    result = []
    for token in gensim.utils.simple_preprocess(tweet):
        if token not in gensim.parsing.preprocessing.STOPWORDS \
                and len(token) > 2:  # drops words with less than 3 characters
            result.append(lemmatize(token))
    return result

def lemmatize(token):
    """Returns lemmatization of a token"""
    return WordNetLemmatizer().lemmatize(token, pos='v')

def preprocess_tweet(tweet):
    """Main master function to clean tweets, stripping noisy characters, and tokenizing use lemmatization"""
    # tweet = remove_users(tweet)
    tweet = remove_links(tweet)
  #  tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
  # tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet_token_list = tokenize(tweet)  # apply lemmatization and tokenization
    tweet = ' '.join(tweet_token_list)
    tweet = re.sub(' amp ', '&', tweet)
    return tweet

def basic_clean(tweet):
    """Main master function to clean tweets only without tokenization or removal of stopwords"""
    # tweet = remove_users(tweet)
    # tweet = remove_hashtags(tweet)
    tweet = remove_av(tweet)
    tweet = tweet.lower()  # lower case
    tweet = re.sub('[' + punctuation + ']+', ' ', tweet)  # strip punctuation
    tweet = re.sub('\s+', ' ', tweet)  # remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet)  # remove numbers
    tweet = re.sub(' & ', '', tweet)
    return tweet

def tokenize_tweets(df):
    df['tokens'] = df.tweet.apply(preprocess_tweet)
    num_tweets = len(df)
    print('Complete. Number of Tweets that have been cleaned and tokenized : {}'.format(num_tweets))
    return df

In [ ]:
df = pd.read_csv("df_nlp_louvain.csv")


In [ ]:
df_filter = df.loc[(df["likes_count"]>=5) & (df["retweets_count"]>= 3)]
df = df_filter

In [ ]:
df = df.drop_duplicates()

In [ ]:
df[df["community"] == "community_3"]

,date,time,name,tweet,mentions,replies_count,retweets_count,likes_count,hashtags,community,original_community_extension
12456,2020-02-26,19:45:30,Ambazonia Prisoners of Conscience Support Network,@TheDailyShow @RepKarenBass @seanogallagher @R...,NaN,3,36,10,"['cbc', 'cameroon', 'feb14', 'massacre', 'amba...",community_3,2189
12457,2020-08-03,20:04:54,Ambazonia Prisoners of Conscience Support Network,@hrw #Corrupt @HRW MISSED #mass #arrest of 100...,"[{'screen_name': 'hrw', 'name': 'human rights ...",0,15,6,"['corrupt', 'mass', 'arrest', 'ambazonians', '...",community_3,2189
12458,2020-08-09,19:53:33,Ambazonia Prisoners of Conscience Support Network,@OchaCameroon @SitiBOussein @OCHAROWCA @UN_Spo...,"[{'screen_name': 'sitiboussein', 'name': 'siti...",0,23,8,"['corrupt', 'french', 'comoros', 'nyc', 'black...",community_3,2189
12459,2020-08-15,18:11:08,Ambazonia Prisoners of Conscience Support Network,@hrw @HRW is at it AGAIN. Just as in #CotedIvo...,"[{'screen_name': 'hrw', 'name': 'human rights ...",0,24,7,"['cotedivoire', 'distraction', 'gaslighting', ...",community_3,2189
12460,2020-08-30,23:58:58,Ambazonia Prisoners of Conscience Support Network,@OchaCameroon @OCHAROWCA @NwswResponse @OchaCa...,"[{'screen_name': 'senateforeign', 'name': 'sen...",2,31,16,"['whitewash', 'coverup', 'crimes', 'against', ...",community_3,2189
...,...,...,...,...,...,...,...,...,...,...,...
173756,2021-02-15,17:02:32,Zehabesha,#Ethiopia is the pride of people of #African D...,"[{'screen_name': 'iamhawknewsome', 'name': 'ha...",4,97,103,"['ethiopia', 'african', 'panafricanist', 'anti...",community_3,2189
173768,2021-02-15,09:39:27,Jennifer Selina,@wdavison10 @EthioHRC @hrcoethio @ahrethiOrg @...,"[{'screen_name': 'blklivesmatter', 'name': 'bl...",0,7,9,"['ethiopia', 'eritrea', 'tigraygenocide']",community_3,2189
173810,2021-02-15,03:16:04,Getachew G. Temare,"#TigrayGenocide, @AngryBlackLady #BlackLivesM...","[{'screen_name': 'angryblacklady', 'name': 'im...",0,9,6,"['tigraygenocide', 'blacklivesmatter', 'ethiop...",community_3,2189
173900,2021-02-17,09:47:29,Iyoba 🇪🇹 🟩🟨🟥 🇪🇹,"@PaulKagame: Let me give you an advice, if you...","[{'screen_name': '_africanunion', 'name': 'afr...",2,8,20,"['tigray', 'africanist', 'unsc', 'blackhistory...",community_3,2189


In [ ]:
df["tweet"] = df.tweet.astype("str")

In [ ]:
import nltk

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df["preprocessed_tweet"] = df["tweet"].apply(preprocess_tweet)


In [ ]:
df.value_counts()

date        time      name                                                tweet                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              mentions                                                                                                                                                                                                                                                                            replies_count  retweets_count  likes_count  hashtags                                                                   

In [ ]:
df.community.value_counts()

community_1      1506
community_0      1377
community_2      1059
community_5       569
community_3       475
                 ... 
community_42        1
community_31        1
community_96        1
community_54        1
community_112       1
Name: community, Length: 87, dtype: int64

In [ ]:
tweets = df.preprocessed_tweet.to_list()
timestamp = df.date.to_list()

In [ ]:
topic_model = BERTopic(language="english", nr_topics = 30)
topics, probs = topic_model.fit_transform(tweets)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,6739,-1_black_police_people_protest
1,0,298,0_resist_thank_tag_resisters
2,1,168,1_shoot_man_kill_police
3,2,137,2_icantbreathe_whiteprivilege_racism_white
4,3,135,3_cameroon_nzajep_french_collycute
5,4,112,4_portland_pdxprotests_wallofmoms_portlandprotest
6,5,110,5_breonnataylor_louisville_lmpd_courierjournal
7,6,95,6_realdonaldtrump_trump_potus_gop
8,7,91,7_matter_live_black_blue
9,8,88,8_taylor_breonna_arrest_brett


In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_barchart()


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)


In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [ ]:
 topic_model.get_representative_docs(0)


['realdonaldtrump racist realdonaldtrump cause current climate civil unrest encourage support use police brutality trumpmeltdown',
 'elect president nah&shirt hard pass',
 'mmpadellan realdonaldtrump future president joebiden medal freedom kaepernick inspire athletes takeaknee support',
 'kinder vernamyers brenebrown medellbriggsmd jacqehoward academicparity charter value drumeshprabhu ayshakhoury annmcairns drjanicegassam thenosm mashalamarie jazdzewskitara googleexpertuk wef bmckinney mindfulacademix womensci whenequality stemhigher stemettes stemwomen centrewj drdanijackson academicdilemma academicvoices kdoctors nhsbme nhsbame doctorsoumya womenspeakershc drericding arghavan salles mcloydkeli memej gbosquesmd repjayapal blklivesmatter civilrightsorg bmj latest special issue reflect work live doctor ethnic minority background healthcare experience ethnic minority patients thercn blm medtwitter medicinetoo ana safavi sdg',
 'drumeshprabhu vernamyers brenebrown medellbriggsmd jacqehow

In [ ]:
df = df.drop_duplicates()

In [ ]:
df["topic_tweet"]= df["preprocessed_tweet"].apply(lambda x: print(x))

In [ ]:
tweet_c0 = df.loc[df['community']=="community_0"]


In [ ]:
tweet_c0_list = df["tweet"].to_list()

In [ ]:
topic_c0 = topic_model.transform(tweet_c0_list)

KeyboardInterrupt: ignored

In [ ]:
topic_c0

# Prove con singole community

In [ ]:
pd.set_option("display.max_rows", None)

In [ ]:
com1 = df[df["community"]=="community_1"]
com0 = df[df["community"]=="community_0"]
com2 = df[df["community"]=="community_2"]
com5 = df[df["community"]=="community_5"]
com3 = df[df["community"]=="community_3"]
com4 = df[df["community"]=="community_4"]
com8 = df[df["community"]=="community_8"]


In [ ]:
tweets = com1.tweet.to_list()
timestamp = com1.date.to_list()

In [ ]:
topic_model = BERTopic(language="english", nr_topics = 10)
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
topic_model.get_topic_info()



In [ ]:
topic_model.visualize_barchart()



In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [ ]:
tweets = com0.tweet.to_list()
timestamp = com0.date.to_list()

In [ ]:
topic_model = BERTopic(language="english", nr_topics = 10)
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name
0,6,1216,6_the_blacklivesmatter_to_and
1,5,49,5_the_to_of_nfl
2,2,31,2_gatherings_banning_tells_large
3,3,23,3_speaks_vote_it_silenced
4,-1,17,-1_loeffler_wnba_gop_viewing
5,0,16,0_in_rigged_opal_bet
6,4,13,4_poetry_poem_micropoetry_haiku
7,1,12,1_financed_supported_machines_infiltration


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


## Community 2

In [ ]:
tweets = com2.preprocessed_tweet.to_list()
timestamp = com2.date.to_list()

In [ ]:
topic_model = BERTopic(language="english", nr_topics = 10)
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,423,-1_black_people_vote_know
1,0,234,0_police_protest_shoot_protesters
2,1,158,1_thank_resist_fbrparty_follow
3,2,68,2_people_white_icantbreathe_whiteprivilege
4,3,38,3_sign_oakland_richmond_today
5,4,29,4_vote_senate_racism_warnock
6,5,28,5_vote_dems_wtpblue_usa
7,6,24,6_icantbreathe_gdamerica_race_pain
8,7,20,7_realdonaldtrump_trump_trumpdeathtoll_sound
9,8,19,8_live_matter_black_response


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


In [ ]:
import pickle

In [ ]:
with open('model_comm_8.pickle', 'wb') as handle:
     pickle.dump(topic_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Community 5

In [ ]:
tweets = com5.preprocessed_tweet.to_list()
timestamp = com5.date.to_list()

In [ ]:
topic_model = BERTopic(language="english", nr_topics = 10)
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
# old: topic_model.get_topic_info()

,Topic,Count,Name
0,-1,222,-1_black_support_live_people
1,0,59,0_people_visitcamberwell_kingsheathltn_rmrstoo
2,1,47,1_bbc_marxist_bbcnews_organisation
3,2,45,2_geffryemustfall_museumofthehome_protest_hack...
4,3,43,3_antiracismday_anti_day_trump
5,4,37,4_black_racism_people_need
6,5,34,5_nyc_nypd_square_park
7,6,26,6_borisjohnson_illegal_boris_tories
8,7,25,7_police_mohamud_justice_begin
9,8,19,8_violent_leave_far_ban


In [ ]:
topic_model.visualize_topics()


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


## Community 3

In [ ]:
tweets = com3.preprocessed_tweet.to_list()
timestamp = com3.date.to_list()

In [ ]:
## runnare per caricare i modelli
topic_model = pd.read_pickle(r'model_comm_3.pickle')
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
# topic_model = BERTopic(language="english", nr_topics = 10)


In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name
0,0,146,0_police_blm_black_protest
1,-1,117,-1_kamalaharris_joebiden_lindat_ablinken
2,1,43,1_cameroon_harmonypartyuk_french_opcameroon
3,2,34,2_technotna_anonymous_anonnewsus_aloyzio
4,3,30,3_back_norwayun_mexonu_stop
5,4,21,4_nigeria_endsars_nigerian_union
6,5,20,5_tigraygenocide_tigray_blklivesmatter_fail
7,6,18,6_tigraygenocide_ethiopia_tigray_eritrea
8,7,17,7_uganda_hebobiwine_senatormenendez_ugandans
9,8,16,8_justice_pass_social_wecantwait


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)


## Community 8

In [ ]:
tweets = com8.preprocessed_tweet.to_list()
timestamp = com8.date.to_list()

In [ ]:
topic_model = BERTopic(language="english", nr_topics = 10)
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
## runnare per caricare i modelli
topic_model = pd.read_pickle(r'model_comm_8.pickle')
topics, probs = topic_model.fit_transform(tweets)

In [ ]:
topic_model.get_topic_info()


,Topic,Count,Name
0,-1,168,-1_black_caption_police_people
1,0,57,0_ice_abolishice_portland_portlandriots
2,6,37,6_black_white_live_racism
3,1,29,1_party_vote_votegreenparty_support
4,3,27,3_portland_portlandoregon_pdx_blm
5,4,19,4_blm_facts_win_powerful
6,5,12,5_breonna_taylor_sayhername_breonnataylor
7,2,11,2_wallofmoms_pdxprotest_pdxundersiege_portlandor


In [ ]:
topic_model.get_topic_info()
## questo è quello con il modello caricato, io boh

,Topic,Count,Name
0,-1,158,-1_black_caption_need_racism
1,2,58,2_wallofmoms_portland_pdx_protest
2,0,57,0_ice_abolishice_portland_portlandriots
3,4,41,4_black_white_live_people
4,5,20,5_blm_facts_peace_win
5,3,14,3_breonna_taylor_sayhername_breonnataylor
6,1,12,1_party_vote_votegreenparty_green


In [ ]:
topics_over_time = topic_model.topics_over_time(tweets, topics, timestamp, nr_bins=20)

topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=10)